In [1]:
import os
import numpy as np
import pandas as pd
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from pytorch_lightning.core.lightning import LightningModule
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, random_split
from torchvision import transforms

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
PARAMS = {
    'units_input': 256,
    'units_in_hidden_layer': 256,
    'num_hidden_layers': 16,
    'activation': 'relu',
    'optimizer': 'adam',
    'lr': 0.0016180877538810258,
    'decay_lr': "ExponentialDecay",
    'kernel_initializer_input': 'normal',
    'kernel_initializer_hidden': 'normal',
    'decay_steps': 20000,
    'decay_rate': 0.3117773360761731,
    'obj_func': 'mape',
    'batch_size': 1024,
    'batch_norm': False
}

In [3]:
features_opt = ['AccumCharge', 'nPMTs', 'R_cc', 'R_cht', 'pe_mean',
                'pe_std', 'pe_skew', 'pe_kurtosis', 'pho_cc', 'pho_cht',
                'ht_2p', 'ht_5p', 'ht_10p', 'ht_15p', 'ht_20p',
                'ht_25p', 'ht_30p', 'ht_35p', 'ht_40p', 'ht_45p',
                'ht_50p', 'ht_55p', 'ht_60p', 'ht_65p', 'ht_70p',
                'ht_75p', 'ht_80p', 'ht_85p', 'ht_90p', 'ht_95p']

In [4]:
def MAPELoss(y_pred, y_true):
    return torch.mean(torch.abs((y_true - y_pred) / y_true))    

In [5]:
class JFCDNN(LightningModule):
    def __init__(self, data_path, real=True):
        super().__init__()

        self.input_layer = nn.Linear(len(features_opt), PARAMS['units_input'])
        self.hidden_layer = nn.Linear(
            PARAMS['units_in_hidden_layer'],
            PARAMS['units_in_hidden_layer']
        )
        self.output_layer = nn.Linear(PARAMS['units_in_hidden_layer'], 1)
        self.real = real
        self.data_path = data_path
        
    def configure_optimizers(self):
        
        optimizer = optim.Adam(self.parameters(), lr=PARAMS['lr'])
        scheduler = optim.lr_scheduler.ExponentialLR(
            optimizer, gamma=0.999708681747017)
        return [optimizer], [scheduler]

    def forward(self, x):
        
        if PARAMS['activation'] == 'relu':
            activation = F.relu
        
        x = self.input_layer(x)
        x = activation(x)
        
        for i in range(PARAMS['num_hidden_layers']):
            x = self.hidden_layer(x)
            x = activation(x)
    
        x = self.output_layer(x)
        return x
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x.float())
        loss_mape = MAPELoss(y_pred, y)
        return loss_mape
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x.float())
        loss_mape = MAPELoss(y_pred, y)
        loss_mse = F.mse_loss(y_pred, y)
        self.log("val_loss", loss_mape, prog_bar=True)
        self.log("val_mse", loss_mse, prog_bar=True)
        return loss_mape
    
    def prepare_data(self):
        if self.real:
            self.dataset = pd.read_csv(
                f'{self.data_path}/ProcessedTrainReal/ProcessedTrain.csv.gz')
        else:
            self.dataset = pd.read_csv(
                f'{self.data_path}/ProcessedTrainIdeal/ProcessedTrain.csv.gz')
        self.dataset = self.dataset[self.dataset['edepR'] < 17.2]
        self.target = self.dataset['edep']
        self.std = self.dataset[features_opt].std()
        self.mean = self.dataset[features_opt].mean()
        self.dataset = (self.dataset[features_opt] - self.mean) / self.std
        self.dataset['edep'] = self.target
        
    def setup(self, stage=None):

        if stage == "fit" or stage is None:
            
            self.data_train, self.data_val = random_split(
                np.array(self.dataset), [4143994, 460444],
            )
            
    def train_dataloader(self):
        return DataLoader(self.data_train, batch_size=PARAMS['batch_size'])

    def val_dataloader(self):
        return DataLoader(self.data_val, batch_size=PARAMS['batch_size'])

In [ ]:
model = JFCDNN()
model.to(device)

In [6]:
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import LearningRateMonitor

num_epochs = 100
seed_everything(22, workers=True)

model = JFCDNN(data_path='/mnt/cephfs/ml_data/mc_2021/processed_data')
trainer = Trainer(
    deterministic=True,
    accelerator="gpu",
    gpus=[0],
    max_epochs=num_epochs,
    callbacks=[
        EarlyStopping(monitor="val_loss", patience=25),
        LearningRateMonitor(),
    ],
)

Global seed set to 22
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
trainer.fit(model)

In [ ]:
models = []
models.append(load_model("../models/5M_ideal.h5"))
models.append(load_model("../models/5M_real.h5"))
models.append(model)

In [ ]:
scalers = []
scalers.append(pickle.load(open('../models/scaler_ideal.pkl','rb')))
scalers.append(pickle.load(open('../models/scaler_real.pkl','rb')))
scalers.append(pickle.load(open('../models/scaler_real.pkl','rb')))

In [ ]:
opt_features = ['AccumCharge', 'nPMTs', 'R_cc', 'R_cht', 'pe_mean',
                'pe_std', 'pe_skew', 'pe_kurtosis', 'pho_cc', 'pho_cht',
                'ht_2p', 'ht_5p', 'ht_10p', 'ht_15p', 'ht_20p',
                'ht_25p', 'ht_30p', 'ht_35p', 'ht_40p', 'ht_45p',
                'ht_50p', 'ht_55p', 'ht_60p', 'ht_65p', 'ht_70p',
                'ht_75p', 'ht_80p', 'ht_85p', 'ht_90p', 'ht_95p',
                'edep']

In [ ]:
%%time
path='/mnt/cephfs/ml_data/mc_2021/'
energies = [0.1, 0.3, 0.6] + list(range(1, 10))
y_true_array = []
y_pred_array = []

for j in tqdm(range(6), "Options..."):
    if j%2==0:
        name = 'ProcessedTestIdeal'
    else:
        name = 'ProcessedTestReal'
    y_true = []
    y_pred = []
    for energy in tqdm(energies, "Energies...", leave=False):
        test = pd.read_csv('{}processed_data/{}/{}MeV.csv.gz'.format(path, name, energy))
        test = test[test['edepR'] < 17.2]
        edep = np.array(test['edep'])
        X_test = test[opt_features].iloc[:, :-1]
        if j <= 1:
            k = 0
        elif j >= 2 and j <= 3:
            k = 1
        else:
            k = 2
        X_test = scalers[k].transform(X_test)
        edep_preds = models[k].predict(X_test).flatten()
            
        y_true.append(edep)
        y_pred.append(edep_preds)
    y_true_array.append(y_true)
    y_pred_array.append(y_pred)

In [ ]:
diffs = np.array([
    [y_pred_array[j][i] - y_true_array[j][i] for i in range(len(y_pred_array[0]))]
    for j in range(len(y_pred_array))
])

In [ ]:
energies = [0.1, 0.3, 0.6] + list(range(1, 10))
energies = np.array([1.022+i for i in energies]).round(5)

In [ ]:
names = [
    'I. model -> I. data', 'I. model -> R. data',
    'R. model -> I. data', 'R. model -> R. data',
    'R. PT model -> I. data', 'R. PT model -> R.data'
]

In [ ]:
pio.templates.default = 'plotly_white'

np.random.seed(22)
for k in range(2):
    fig = go.Figure()
    for i in range(len(diffs[k])): 
        indx = np.random.randint(0, diffs[k][i].shape[0], 10000)
        mini = min(diffs[k][i][indx].min(), diffs[k+2][i][indx].min(), diffs[k+4][i][indx].min())
        maxi = max(diffs[k][i][indx].max(), diffs[k+2][i][indx].max(), diffs[k+4][i][indx].max())
        size = (maxi - mini) / 100
        
        fig.add_trace(
            go.Histogram(
                x=diffs[k][i][indx],
                xbins=dict(
                    start=mini,
                    end=maxi,
                    size=size
                ),
                marker=dict(
                    opacity=0.8,
                    color='darkred'
                ),
                name=names[k],
                histnorm='probability density',
                visible=(i==0)
            )
        )
        
    for i in range(len(diffs[k])): 
        indx = np.random.randint(0, diffs[k][i].shape[0], 10000)
        mini = min(diffs[k][i][indx].min(), diffs[k+2][i][indx].min(), diffs[k+4][i][indx].min())
        maxi = max(diffs[k][i][indx].max(), diffs[k+2][i][indx].max(), diffs[k+4][i][indx].max())
        size = (maxi - mini) / 100

        fig.add_trace(
            go.Histogram(
                x=diffs[k+2][i][indx],
                xbins=dict(
                    start=mini,
                    end=maxi,
                    size=size
                ),
                marker=dict(
                    opacity=0.8,
                    color='royalblue'
                ),
                name=names[k+2],
                histnorm='probability density',
                visible=(i==0)
            )
        )

    for i in range(len(diffs[k])): 
        indx = np.random.randint(0, diffs[k][i].shape[0], 10000)
        mini = min(diffs[k][i][indx].min(), diffs[k+2][i][indx].min(), diffs[k+4][i][indx].min())
        maxi = max(diffs[k][i][indx].max(), diffs[k+2][i][indx].max(), diffs[k+4][i][indx].max())
        size = (maxi - mini) / 100

        fig.add_trace(
            go.Histogram(
                x=diffs[k+4][i][indx],
                xbins=dict(
                    start=mini,
                    end=maxi,
                    size=size
                ),
                marker=dict(
                    opacity=0.8,
                    color='darkgreen'
                ),
                name=names[k+4],
                histnorm='probability density',
                visible=(i==0)
            )
        )


    buttons = []
    for N in range(len(diffs[k])): 
        buttons.append(
            dict(
                 args=['visible', [False]*N + [True] + [False]*(len(diffs[k])-1-N)],
                     label='Energy =  {} MeV'.format(energies[N]),
                 method='restyle'
            )
        )
        
    fig.update_layout(

        xaxis = dict(
            showline=True,
            ticks='outside',
            mirror=True,
            linecolor='black',
            showgrid=True,
            gridcolor='grey',
            gridwidth=0.25,
        ),

        yaxis = dict(
            showline=True,
            ticks='outside',
            mirror=True,
            linecolor='black',
            tick0=0,
            showgrid=True,
            gridcolor='grey',
            gridwidth=0.25,
            zeroline=True,
            zerolinecolor='black',
            zerolinewidth=0.25
            ),
    )

    fig.update_layout(
        xaxis_title=r"$$E_{rec} - E_{true}$$",
        barmode='overlay',
        updatemenus=list([
            dict(
                x=0.3,
                y=1.2,
                yanchor='top',
                buttons=buttons
            ),
        ]),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.05,
            xanchor="right",
            x=1
        )
    )
    
    fig.show()